In [1]:
import numpy as np
import pandas as pd

In [27]:
# let's define the code



def entropy(y):
    """
    Information Entropy
    """
    hist = np.bincount(y)
    ps = hist / np.sum(hist)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])


def gini(y):
    """
    Gini Impurity
    """
    hist = np.bincount(y)
    N = np.sum(hist)
    return 1 - sum([(i / N) ** 2 for i in hist])


class Node:
    def __init__(self, left, right, rule):
        self.left = left
        self.right = right
        self.feature = rule[0]
        self.threshold = rule[1]


class Leaf:
    def __init__(self, value):
        """
        `value` is an array of class probabilities if classifier is True, else
        the mean of the region
        """
        self.value = value


class DecisionTree:
    def __init__(
        self,
        criterion = "entropy",
        n_feats = None
    ):
        self.criterion = criterion
        self.depth = 0
        self.max_depth = np.inf

    def fit(self, X, Y):
        """
        Fit a binary decision tree to a dataset.
        Parameters
        """
        # determine how many features we consider for each splits (n_feats)
        self.n_feats = X.shape[1]
        self.root = self._grow(X, Y)


        
    def _grow(self, X, Y, cur_depth=0):
        # if all labels are the same, return a leaf
        if len(set(Y)) == 1:
            return Leaf(Y[0])

        cur_depth += 1
        self.depth = max(self.depth, cur_depth)

        N, M = X.shape

        # generate a random sample from the 1-D array of M columns without replacement
        feat_idxs = np.random.choice(M, size = self.n_feats, replace = False)

        # greedily select the best split according to `criterion`
        # apply segment function
        feat, thresh = self._segment(X, Y, feat_idxs)
        l = np.argwhere(X[:, feat] <= thresh).flatten()
        r = np.argwhere(X[:, feat] > thresh).flatten()

        # apply the same as before one levael down in the tree
        # grow the children that result from the split
        left = self._grow(X[l, :], Y[l], cur_depth)
        right = self._grow(X[r, :], Y[r], cur_depth)
        return Node(left, right, (feat, thresh))

    def _segment(self, X, Y, feat_idxs):
        """
        Find the optimal split rule (feature index and split threshold) for the
        data according to `self.criterion`.
        """
        best_gain = -np.inf
        split_idx, split_thresh = None, None
        for i in feat_idxs:
            # get the i-th column which we created before with the number generator
            vals = X[:, i]
            levels = np.unique(vals)
            # [:-1] -> take all but the last element
            # [1:] -> take all but the first element
            thresholds = (levels[:-1] + levels[1:]) / 2 if len(levels) > 1 else levels
            # now call function impurity gain
            # collect all values of the gain in list (for each threshold)
            gains = np.array([self._impurity_gain(Y, t, vals) for t in thresholds])

            # update the gain if new one is btter
            if gains.max() > best_gain:
                split_idx = i
                best_gain = gains.max()
                # save the threshold for the max gain
                split_thresh = thresholds[gains.argmax()]

        return split_idx, split_thresh

    def _impurity_gain(self, Y, split_thresh, feat_values):
        """
        Compute the impurity gain associated with a given split.
        IG(split) = loss(parent) - weighted_avg[loss(left_child), loss(right_child)]
        """
        if self.criterion == "entropy":
            loss = entropy
        elif self.criterion == "gini":
            loss = gini

        # calculate the loss of the parent before the split
        parent_loss = loss(Y)

        # generate splits by getting all index positions where the value is <= or > than the threshold
        left = np.argwhere(feat_values <= split_thresh).flatten()
        right = np.argwhere(feat_values > split_thresh).flatten()

        if len(left) == 0 or len(right) == 0:
            return 0

        # compute the weighted avg. of the loss for the children
        n = len(Y)
        # number of indices making the threshold
        n_l, n_r = len(left), len(right)
        
        # calculate loss according evaluation criterion defined
        # taking the rows by left and right index of Y as the input
        e_l, e_r = loss(Y[left]), loss(Y[right])
        child_loss = (n_l / n) * e_l + (n_r / n) * e_r

        # impurity gain is difference in loss before vs. after split
        ig = parent_loss - child_loss
        return ig



    def _traverse(self, X, node, prob=False):
        print('x', X)
        if isinstance(node, Leaf):
            print(node.value)
            return node.value
        if X[node.feature] <= node.threshold:
            print('feature:',node.feature)
            print('threshold',node.threshold)
            #print('node left',node.left)
            #print('node right',node.right)
            return self._traverse(X, node.left, prob)
        return self._traverse(X, node.right, prob)

    
    def predict(self, X):
        """
        Use the trained decision tree to classify or predict the examples in `X`.
        """
        # iterates over rows of X!
        return np.array([self._traverse(x, self.root) for x in X])


In [28]:
dt = DecisionTree()
X = np.array([[2,-1,5,6],[4,-2,3,0],[3,0,1,5], [-1,1,2,2]])
Y = np.random.choice(10,4)
print(Y)
dt.fit(X,Y)
dt.predict(X)

[2 8 5 9]
x [ 2 -1  5  6]
x [ 2 -1  5  6]
feature: 0
threshold 3.0
x [ 2 -1  5  6]
2
x [ 4 -2  3  0]
x [ 4 -2  3  0]
x [ 4 -2  3  0]
8
x [3 0 1 5]
feature: 2
threshold 2.5
x [3 0 1 5]
x [3 0 1 5]
5
x [-1  1  2  2]
feature: 2
threshold 2.5
x [-1  1  2  2]
feature: 0
threshold 1.0
x [-1  1  2  2]
9


array([2, 8, 5, 9])